<a href="https://colab.research.google.com/github/MoChiaoKai/Kai0214/blob/main/0502_Ask_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk google-generativeai

In [ ]:
from google.colab import userdata
from flask import Flask, request, abort
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok


def ngrok_start():
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(ngrok_token)
    ngrok.connect(5000)
    run_with_ngrok(app)

from linebot.v3 import WebhookHandler

from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

Configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

import google.generativeai as gemini

gemini.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = gemini.GenerativeModel('gemini-2.0-flash-exp')
def ask_gemini(prompt):
    response = model.generate_content(prompt)
    return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    model = gemini.GenerativeModel('gemini-2.0-flash-exp')
    response = model.generate_content(event.message.text)
    return_response = response.text

    with ApiClient(Configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=return_response)]
            )
        )

ngrok_start()

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e863-34-75-151-147.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [27/May/2025 08:54:01] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 08:54:09] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 08:55:10] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2025 08:56:39] "POST /callback HTTP/1.1" 200 -


In [ ]:
import google.generativeai as gemini
from google.colab import userdata

# 確保使用您的 API Key
gemini.configure(api_key=userdata.get("GOOGLE_API_KEY"))

print("目前可用的 Gemini 模型清單：")
for model in gemini.list_models():
    # 只列出支援 generateContent 的模型
    if 'generateContent' in model.supported_generation_methods:
        print(model.name)

目前可用的 Gemini 模型清單：
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
m